In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.patches import Wedge
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable, Reds

plt.rcParams['font.family'] = 'Arial'
plt.rcParams['axes.unicode_minus'] = False 



xls_new = pd.ExcelFile(r'LAI_all.xlsx')


dfs_new = [xls_new.parse(x) for x in xls_new.sheet_names]


def check_recovery(row):

    for i in range(len(row)):
        if (row[i:i+2] >= 0).all():
            return 1
            break


    for i in range(4,len(row) - 3):
        if (row[i:i+4] < 0).all() and (row[i:i+4].diff().sum()  <= 0):
            return 3
            break

    return 2

for i, df in enumerate(dfs_new[:2], start=1):

    lon=df['lon']
    lat=df['lat']
    print(i)    

    df = df.iloc[:, -8:].sub(df.iloc[:, -9], axis=0)

    
    df['Recovery'] = df.apply(check_recovery, axis=1).astype(int)
    

    
    df['lon']=lon
    df['lat']=lat
    
    fig = plt.figure(figsize=(5, 5), dpi=600)
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_global()


    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.LAND)


    for index, row in df.iterrows():
        lon = row['lon']
        lat = row['lat']
        Recovery = row['Recovery']

        if Recovery == 1:
            color = 'darkgreen'
        elif Recovery == 2:
            color = 'lightgreen'
        elif Recovery == 3:
            color = 'red'       

        ax.scatter(lon, lat, color=color, s=0.6, marker='o', transform=ccrs.PlateCarree()) 
    

    recovery_counts = df['Recovery'].value_counts()

    sizes = [recovery_counts.get(1, 0), recovery_counts.get(2, 0), recovery_counts.get(3, 0)]
    colors = ['darkgreen', 'lightgreen', 'red']


    def autopct_format(pct):
        return f'{pct:.1f}%' if pct > 0 else ''

    inset_ax = fig.add_axes([0.13, 0.32, 0.20, 0.23])  # [left, bottom, width, height]
    wedges, texts, autotexts = inset_ax.pie(sizes, colors=colors, autopct=autopct_format, startangle=90)
    for autotext in autotexts:
        autotext.set_fontsize(10)
    
        x, y = autotext.get_position()
        x = 1.3 * x
        y = y

        autotext.set_position((x, y))
    
    inset_ax.axis('equal') 
    
    
    plt.show()
    